Faço as importações necessárias

In [1]:
import requests
from bs4 import BeautifulSoup
import io
from os import system

Acesso o endpoint de notícias do site do Flamengo

In [2]:
host = "https://www.flamengorj.com.br"
MAX_BANNERS_PAGES = 10

responses = []
for i in range(1,MAX_BANNERS_PAGES+1):
    url = host + "/noticias"
    if i > 1:
        url = url + f'/{i}'
    r = requests.get(url)
    
    if r.status_code == 200:
        responses.append(r)
    elif r.status_code == 404:
        break
    

Pego o conteúdo html recebido

In [3]:
htmls = [] 
for r in responses:
    htmls.append(r.content)

Pego os banners de notícias dentro dos htmls 

In [4]:
banners = []
for html in htmls:
    bs = BeautifulSoup(html)
    banners_page = bs.find_all("div",class_="post-grid__item col-12 col-md-6")
    for bpage in banners_page:
        banners.append(bpage)

qtd_noticias = len(banners)
print("Quantidade de notícias capturadas:",qtd_noticias)

Quantidade de notícias capturadas: 600


Pego os links para cada notícia capturada

In [5]:
links = []
for bn in banners:
  links.append(bn.find("a")['href'])

print(links)

['/noticia/flamengo-descarta-novas-contratacoes-para-priorizar-compras-de-thiago-maia-e-pedro', '/noticia/pedro-esta-a-um-gol-de-se-igualar-a-thiago-galhardo-na-artilharia-do-brasil-em-2020', '/noticia/hugo-souza-demonstra-vontade-de-permenecer-no-flamengo-e-encaminha-renovacao', '/noticia/catarse-novo-documentario-sobre-final-da-libertadores-sera-lancado-no-aniversario-de-1-ano-do-titulo', '/noticia/alem-da-lideranca-fla-busca-vitoria-sobre-o-atletico-mg-para-abrir-vantagem-no-retrospecto-historico', '/noticia/marcos-braz-revela-bastidores-das-negociacoes-de-thiago-maia', '/noticia/dirigente-do-flamengo-diz-compreender-erros-mas-alerta-cobramos-superacao', '/noticia/dome-revela-conversa-franca-com-jogadores-sobre-erros-individuais', '/noticia/dirigente-do-flamengo-destaca-evolucao-de-matheuzinho-tem-muito-potencial', '/noticia/matheuzinho-tem-assistencias-nas-tres-competicoes-que-o-flamengo-disputa', '/noticia/ele-voltou-gabigol-se-prepara-para-jogo-contra-o-atletico-mg-e-da-aviso-a-t

Trato cada link deixando-os completos

In [6]:
for i in range(len(links)):
  links[i] = host + links[i]

print(links)

['https://www.flamengorj.com.br/noticia/flamengo-descarta-novas-contratacoes-para-priorizar-compras-de-thiago-maia-e-pedro', 'https://www.flamengorj.com.br/noticia/pedro-esta-a-um-gol-de-se-igualar-a-thiago-galhardo-na-artilharia-do-brasil-em-2020', 'https://www.flamengorj.com.br/noticia/hugo-souza-demonstra-vontade-de-permenecer-no-flamengo-e-encaminha-renovacao', 'https://www.flamengorj.com.br/noticia/catarse-novo-documentario-sobre-final-da-libertadores-sera-lancado-no-aniversario-de-1-ano-do-titulo', 'https://www.flamengorj.com.br/noticia/alem-da-lideranca-fla-busca-vitoria-sobre-o-atletico-mg-para-abrir-vantagem-no-retrospecto-historico', 'https://www.flamengorj.com.br/noticia/marcos-braz-revela-bastidores-das-negociacoes-de-thiago-maia', 'https://www.flamengorj.com.br/noticia/dirigente-do-flamengo-diz-compreender-erros-mas-alerta-cobramos-superacao', 'https://www.flamengorj.com.br/noticia/dome-revela-conversa-franca-com-jogadores-sobre-erros-individuais', 'https://www.flamengorj.

Crio a função que me ajudará a pegar o conteúdo de cada notícia em passos posteriores

In [7]:
def get_new(new_):
    title = new_.find('h2',class_='post__title').text
    time = new_.find('time').text
    content = new_.find_all('p')

    for i in range(len(content)-1,-1,-1):
        if content[i].text == '' or content[i].find('span') != None or len(content[i].text) <= 20:
            del content[i]
        else:
            content[i] = str(content[i].text)
    
    return {'title':title,'content':content,'time':time}

Crio a função responsável por capturar cada uma das notícias via links passados

In [8]:
def get_all_news(links):
    news = []
    for link in links:
        r = BeautifulSoup(requests.get(link).content)
        new_ = r.find('div',class_='card__content')
        post = get_new(new_)
        news.append(post)
    return news

Utilizo as funções acima definidas para fazer o scrapping do conteúdo de cada uma das notícias

In [9]:
all_news = get_all_news(links)
print(all_news)

[{'title': 'Flamengo descarta novas contratações para priorizar compras de Thiago Maia e Pedro', 'content': ['Na tarde desta quinta-feira (05), o vice-presidente de futebol do Flamengo, Marcos Braz, concedeu entrevista coletiva no Ninho do Urubu e tratou de diversos temas importantes sobre o clube. Dentre os assuntos abordados, o dirigente foi perguntado sobre a possibilidade de novos reforços chegarem, e afirmou que não pensa nisso neste momento. Pontuou que a prioridade é a resolução dos jogadores que estão no Fla por empréstimo.', '– O Flamengo não falará de reforços nesse momento. A prioridade é a resolução dos jogadores que estão emprestados e tem opção de compra. Depois disso, com calma, vamos ao mercado. Agora nesse momento não estamos tentando ninguém, é tudo mentira, não procede! Estamos sempre analisando, até porque existe um departamento para isso, mas não iremos ao mercado nesse momento.', 'Lembrando que os emprestados ao Flamengo são: Pedro, Pedro Rocha e Thiago Maia. O pr

Crio a função que salva todas as notícias em arquivos .txt na minha máquina

In [10]:
def save_news(all_news):
    system("mkdir crawled_news")
    for n in all_news:
       with  io.open('crawled_news/' + str(id(n)) + '.txt' , 'w',encoding='utf-8')  as file:
        file.write('[TITLE] >> ' + n['title'] + '\n')
        file.write('[TIME] >> ' + n['time'] + '\n')
        file.write('[CONTENT]\n')
        for p in n['content']:
            file.write('[PARAGRAPH]\n')
            file.write(p + '\n')

Uso a função definida acima para salvar as notícias crawleadas

In [11]:
save_news(all_news)